# Network analysis for coauthors - network through time

This looks at how the network changes through time with network metrics.

In [1]:
%load_ext autoreload
%autoreload 2

from src import util as u

import pandas as pd
from functools import reduce
import operator
import networkx as nx

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
nodes = pd.read_csv(u.fn_nodes)

In [3]:
# Import author data with dates
start = 1758
end = 2018
years = end-start
timepoints = 4
interval = round(years/3, 0)
year1= start + interval*1
year2= start + interval*2

In [4]:
print(start, year1)
print(year1, year2) # postal mail invented
print(year2, end)   # airlines invested

1758 1845.0
1845.0 1932.0
1932.0 2018


In [5]:
auth = u.get_spp_df()
auth = auth.drop(columns=['status'])


auth1 = auth[(auth['date'] >= start) & (auth['date']  < year1)][['idx', 'full_name_of_describer']]
auth2 = auth[(auth['date'] >= year1) & (auth['date']  < year2)][['idx', 'full_name_of_describer']]
auth3 = auth[(auth['date'] >= year2) & (auth['date']  <= end)][['idx', 'full_name_of_describer']]

df_li =  [auth1, auth2, auth3]
df_li = [x.groupby('idx')['full_name_of_describer'].apply(lambda x: "%s" % '; '.join(x)) for x in df_li]

/home/ejysoh/miniconda3/envs/msc/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (22,30,60,61,65,86,89,106,117) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/mnt/c/_dev/msc/thesis/2020-08-31-jsa-type-v2-ch3-coauth/src/util.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  spp = spp[~spp['duplicated']][


In [6]:
# Get pairs

def li_pairs(source):
    source = str.split(source, "; ")
    result = []
    N = len(source)
    if (N <= 1):
        return [(source[0], None)]
    else:

        for p1 in range(N):
            for p2 in range(p1+1,len(source)):
                    result.append((source[p1],source[p2]))
        return result


In [7]:
print(df_li[2].iloc[2])
print(li_pairs(df_li[2].iloc[2]))
print(li_pairs(df_li[2].iloc[63]))

Ricardo Ayala Barajas; Michael Scott Engel
[('Ricardo Ayala Barajas', 'Michael Scott Engel')]
[('Earle Gorton Linsley', 'John Winslow MacSwain')]


In [8]:
df_li = [x.apply(lambda x: li_pairs(x)) for x in df_li]

In [9]:
li = [[] for i in range(3)]
for i in range(len(df_li)):
    df = df_li[i]
    for j, row in df.iteritems():
        li[i].append(row[0])

In [10]:
df_li = [pd.DataFrame(x, columns=['p1', 'p2']) for x in li]
df_li = [pd.DataFrame(x).groupby(['p1', 'p2']).size() for x in df_li]
df_li = [x.reset_index() for x in df_li]

In [11]:
def get_edges(df):
    edges = []
    for i, row in df.iterrows():
        edges.append((row.p1, row.p2, row[0]))
    return edges

In [12]:
df_li = [get_edges(df) for df in df_li]

In [13]:
def get_node_names(edges):
    all_node_names = []
    all_node_names = [all_node_names + [e[0], e[1]] for e in edges]
    all_node_names = reduce(operator.add, all_node_names)
    return set(all_node_names)

In [14]:
node_names = [get_node_names(df) for df in df_li]

In [15]:
# Load into graph
g_li = [nx.Graph() for x in range(3)]
for i in range(len(df_li)):

    g_li[i].add_nodes_from(node_names[i])
    g_li[i].add_weighted_edges_from(df_li[i])

In [16]:
for i in range(len(df_li)):
    print("Network", i)
    
    density = nx.density(g_li[i])
    print("Network density:", round(density*100, 1), "%")
    
    triadic_closure = nx.transitivity(g_li[i])
    print("Triadic closure:", round(triadic_closure*100, 1), "%")
    
    # Get subgraphs
    subgraphs = [c for c in sorted(nx.connected_components(g_li[i]), key=len, reverse=True)]
    print("Number of subgraphs:", len(subgraphs))
    
    # Largest component
    components = nx.connected_components(g_li[i])
    largest_component = max(components, key=len) # max number of nodes

    # Create a "subgraph" of just the largest component
    # Then calculate the diameter of the subgraph, just like you did with density.
    subgraph = g_li[i].subgraph(largest_component)
    diameter = nx.diameter(subgraph)
    print("Network diameter of largest component:", diameter)
    
    print([v[0] for v in g_li[i].nodes(data=True)])
    
    
    print("\n")

Network 0
Network density: 33.3 %
Triadic closure: 0 %
Number of subgraphs: 2
Network diameter of largest component: 1
['[Jean-Pierre Omer Anne] Édouard Perris', 'Amédée Louis Michel le Peletier, comte de Saint-Fargeau', 'Jean Guillaume Audinet-Serville', 'Léon [Jean Marie] Dufour']


Network 1
Network density: 4.3 %
Triadic closure: 0 %
Number of subgraphs: 10
Network diameter of largest component: 3
['Beulah Hix Blair', 'Louis Jérôme Reiche', 'Jessie E. Casad', 'Theodore Dru Alison Cockerell', 'James Fletcher', 'Joseph Charles Bequaert', 'Myron Harmon Swenk', 'Franz von Wagner', 'Karl Wilhelm von Dalla Torre', 'Lucy Evelyn Cheesman', 'Charles H. Hicks', 'G. Trautmann', 'Philip Hunter Timberlake', 'Heinrich Friedrich August Karl Ludwig Friese', 'Otto Emil Plath', 'Woldemar Trautmann', 'Edward Sharpe Gaige Titus', 'Hans Bischoff', 'Norma Ebolie LeVeque', 'Sievert Allen Rohwer', 'Grace Adelbert Sandhouse', 'Léon Marc Herminie Fairmaire', 'Ruth Sumner', 'Kurt [Curt] Schrottky', 'Hans [Fr